In [43]:
import pandas as pd
# import os
# os.getcwd()

Read json file in

In [44]:
vt = pd.read_json('review-Vermont_10.json', lines = True)

In [45]:
print(vt.shape)
print(vt.columns)
print(vt.head())

(324725, 8)
Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')
        user_id             name           time  rating  \
0  1.180269e+20    rebecca kerns  1620085852324       5   
1  1.015327e+20    Peter DeForge  1580309946474       5   
2  1.154041e+20    Chad Goulette  1605195974445       5   
3  1.047893e+20  Mark LaFountain  1593005848256       5   
4  1.089807e+20           Jeff R  1582059996120       5   

                                                text  pics  \
0      Always done right from wood stove to screens!  None   
1  A great company to work with.  Their sales and...  None   
2  Great place to do business with staff was grea...  None   
3  Awesome Customer service, quick response, and ...  None   
4  If you need a top quality job, by a group of p...  None   

                                                resp  \
0  {'time': 1620087641504, 'text': 'Good Evening,...   
1  {'time': 1580320228721, 'text': 'Good Afternoo...

Drop 'name' column, convert all text to lower case, drop duplicates (~3000 duplicates)

In [46]:
vt = vt.drop(['name'], axis = 1)
vt.columns = vt.columns.str.lower() 
vt = vt.drop_duplicates(subset=['user_id', 'text', 'gmap_id'])
print(vt.shape)

(321483, 7)


Change the format of the 'pics' column, merge multiple dictionaries into one with list of pic urls

In [47]:
# vt_pics = vt[vt['pics'].notna()]
# print(vt_pics.head())
# print(vt_pics['pics'])
# import pprint
# pprint.pprint(vt_pics.loc[17, 'pics'])

In [48]:
def collapse_pics(pic_list):
    if not pic_list:
        return []  
    urls = []
    for pic_dict in pic_list:
        urls.extend(pic_dict.get('url', []))
    return urls

vt['pics_collapsed'] = vt['pics'].apply(collapse_pics)

print(vt['pics_collapsed'].iloc[17])

['https://lh5.googleusercontent.com/p/AF1QipPQa7V_Ibv9GfDZQXPeNGV3W_dY-rRCJhTYieY9=w150-h150-k-no-p', 'https://lh5.googleusercontent.com/p/AF1QipOX0LxkfikC3U2TFyXJaOB308ZAmCpJ7D0PFMdj=w150-h150-k-no-p']


Separate dataset into vt1 (rating only) and vt2 (everything else)

In [49]:
vt1 = vt[vt['pics'].isna() & vt['text'].isna()] # rating only
vt2 = vt[~vt.index.isin(vt1.index)] # everything else

print(vt1.shape)
print(vt2.shape)

(145565, 8)
(175918, 8)


Average rating

In [50]:
rating_counts = vt.groupby(['gmap_id', 'rating']).size().unstack(fill_value=0)
rating_counts['average'] = vt.groupby('gmap_id')['rating'].mean()
rating_counts = rating_counts[[1, 2, 3, 4, 5, 'average']]
print(rating_counts.head())

rating                                  1   2   3   4    5   average
gmap_id                                                             
0x4054251f453dde0d:0x6fb60c932a35686    0   0   1   6   12  4.578947
0x405426dbc2683d73:0x5540daa9269c9598   1   1   1   6   18  4.444444
0x4056961885a0f575:0x9a02c2199f695127   0   2   0   4   17  4.565217
0x40569854833a734b:0xa0aa4d4cc55b5ac   12  10  38  99  113  4.069853
0x405726108eb74aa1:0x7acdaadaff59d35c   5   0   1   5   34  4.400000


In [34]:
!pip install -q -U transformers torch huggingface_hub python-dotenv


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [57]:
from transformers import pipeline
import requests
from PIL import Image
from io import BytesIO

ocr_pipeline = pipeline("image-to-text", model="microsoft/trocr-base-printed")  

config.json: 0.00B [00:00, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:2199: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use mps:0


In [58]:
vt2 = vt2.copy()

def extract_urls(pic_list):
    if not pic_list:  # handles None or empty
        return []
    urls = []
    for pic in pic_list:
        urls.extend(pic.get('url', []))
    return urls

# Create the urls column safely
vt2['urls'] = vt2['pics'].apply(extract_urls)

def hf_ocr_from_url_list(url_list):
    text_all = ""
    for url in url_list:
        try:
            # call your OCR pipeline here
            text = ocr_pipeline(url)
            text_all += " " + text
        except Exception as e:
            print(f"Error reading image: {url}, {e}")
    return text_all

# Check
print(vt2[['pics', 'urls']].head())

   pics urls
0  None   []
1  None   []
2  None   []
3  None   []
4  None   []


In [ ]:
def ocr_safe(urls):
    if not urls:
        return ""
    return hf_ocr_from_url_list(urls)

for i in range(0, len(vt2), 2):
    vt2.loc[i:i+49, 'image_text'] = vt2.loc[i:i+49, 'urls'].apply(ocr_safe)

Error reading image: https://lh5.googleusercontent.com/p/AF1QipPQa7V_Ibv9GfDZQXPeNGV3W_dY-rRCJhTYieY9=w150-h150-k-no-p, isin_Tensor_Tensor_out only works on floating types on MPS for pre MacOS_14_0. Received dtype: Long
Error reading image: https://lh5.googleusercontent.com/p/AF1QipOX0LxkfikC3U2TFyXJaOB308ZAmCpJ7D0PFMdj=w150-h150-k-no-p, isin_Tensor_Tensor_out only works on floating types on MPS for pre MacOS_14_0. Received dtype: Long
Error reading image: https://lh5.googleusercontent.com/p/AF1QipPQa7V_Ibv9GfDZQXPeNGV3W_dY-rRCJhTYieY9=w150-h150-k-no-p, isin_Tensor_Tensor_out only works on floating types on MPS for pre MacOS_14_0. Received dtype: Long
Error reading image: https://lh5.googleusercontent.com/p/AF1QipOX0LxkfikC3U2TFyXJaOB308ZAmCpJ7D0PFMdj=w150-h150-k-no-p, isin_Tensor_Tensor_out only works on floating types on MPS for pre MacOS_14_0. Received dtype: Long
Error reading image: https://lh5.googleusercontent.com/p/AF1QipPQa7V_Ibv9GfDZQXPeNGV3W_dY-rRCJhTYieY9=w150-h150-k-no-p, 